In [6]:
try:
    import chemiscope
except ImportError:
    !pip3 install chemiscope
    import chemiscope

try:
    import dscribe
except ImportError:
    !pip3 install dscribe
    
from dscribe.descriptors import MBTR
from dscribe.kernels import REMatchKernel
from sklearn.preprocessing import normalize
from sklearn.decomposition import KernelPCA
from sklearn.metrics.pairwise import cosine_similarity

import numpy as np
from ase.db import connect
import math
from utils import *

In [17]:
orientation='100'
calculate_kernel = False
if calculate_kernel:
    all_systems_rows = []
    frames = []
    for t, term in enumerate(termination_types[orientation]):
        this_db = connect('2dpv_set_'+orientation+'_'+term+'.db')
        for thick in [3, 5, 7, 9]:
            for i in range(len(A_site_list)):
                for a in A_site_list[i]:
                    for b in B_site_list[i]:
                        for c in C_site_list[i]:
                            system_name = a + b + c

                            uid = system_name + '3_' + str(orientation) + "_" + str(term) + "_" + str(thick)
                            row = None
                            row = this_db.get(selection=[('uid', '=', uid)])

                            all_systems_rows.append(row)
                            frames.append(row.toatoms())

    all_desc = [row_to_mbtr_descriptor(row) for row in all_systems_rows]
    kernel_matrix = [[1.0 for _ in range(len(all_desc))] for _ in range(len(all_desc))]

    for i in range(len(all_desc) - 1):
        for j in range(i + 1, len(all_desc)):
            this_kernel = cosine_similarity(all_desc[i].reshape(-1, 1), all_desc[j].reshape(-1, 1))[0][0]
            kernel_matrix[i][j] = this_kernel
            kernel_matrix[j][i] = this_kernel
else:
    import pickle
    kernel = pickle.load(open('similarity_kernel_'+orientation+'_mbtr.bp','rb'))
    frames = [i.toatoms() for i in kernel['structures']]
    kpca_1 = kernel['kpca1']
    kpca_2 = kernel['kpca2']

In [24]:
pca = np.array([kpca_1,kpca_2]).T
print(np.shape(pca))
properties = {
    "PCA": pca,
    "termination": kernel['termination'],
    "energy": kernel['energy'],
    "tolerance_factor": kernel['tolerance_factor']
}
widget = chemiscope.show(frames, properties)
widget


(1696, 2)


ChemiscopeWidget(value=None, data='{"meta": {"name": " "}, "structures": [{"size": 8, "names": ["F", "F", "F",…